In [ ]:
#|hide
#|default_exp core

# core

> Fast scripts using daemon mode

In [ ]:
#|export
import socket
from contextlib import redirect_stdout
from io import StringIO
from multiprocessing import get_context
from socketserver import TCPServer, StreamRequestHandler

from fastcore.meta import *
from fastcore.net import *
from fastcore.script import *
from fastcore.utils import *

In [ ]:
#|hide
from nbprocess.showdoc import *

In [ ]:
import time

from fastcore.test import *

In [ ]:
#|export
def send_recv(s, port, host=None, dgram=False, encoding='utf-8'):
    "Wraps `start_client`; send string `s` in `encoding` and return its response"
    with start_client(port, host=host, dgram=dgram) as client:
        client.sendall((s+'\n').encode(encoding))
        with client.makefile('rb') as f: return f.read().decode('utf-8')

In [ ]:
#|export
def _handle(cmd, data):
    "Execute `cmd` with args parsed from `data` and return `stdout`"
    argv = data.decode().strip()
    sys.argv = [cmd.__name__] + (argv.split(' ') if argv else [])
    with redirect_stdout(StringIO()) as s: cmd()
    return s.getvalue().encode()

In [ ]:
#|export
class DaemonHandler(StreamRequestHandler):
    "Execute server's `cmd` with request args using server's process pool"
    def handle(self):
        data = self.rfile.readline().strip()
        future = self.server.pool.submit(_handle, self.server.cmd, data)
        result = future.result()
        self.wfile.write(result)

`DaemonHandler`'s primary use-case is in `DaemonServer`, however it supports any `socketserver.BaseServer` that has `pool` and `cmd` attributes.

In [ ]:
#|export
class DaemonServer(TCPServer): # TODO: could be a mixin to support other servers; `Pool(ed)Server`?
    "A `TCPServer` that executes `cmd` with request args using a process pool"
    @delegates(TCPServer)
    def __init__(self, server_address, cmd, RequestHandlerClass=DaemonHandler, timeout=None, **kwargs):
        self.cmd = cmd # TODO: is this the best place for `cmd`?
        if timeout is not None: self.timeout = timeout
        self.allow_reuse_address = True
        super().__init__(server_address, RequestHandlerClass)
        
    def server_activate(self):
        self.pool = ProcessPoolExecutor(mp_context=get_context('fork')) # TODO: make ctx configurable?
        super().server_activate()
        
    def server_close(self):
        if hasattr(self,'pool'): self.pool.shutdown()
        super().server_close()
        
    def handle_timeout(self): return True

Here's an example of how to use `DaemonServer`. First, define the `cmd`. It should have no arguments itself but rather parse its arguments from `sys.argv`. Its return value isn't used, instead it should write to `stdout`.

In [ ]:
def _cmd():
    import sys
    name = sys.argv[1]
    print(f'Hello, {name}!')

Then start the server. We start it with `handle_request` in a separate thread:

In [ ]:
addr = host,port = 'localhost',9999
@threaded
def _f():
    with DaemonServer(addr, _cmd) as srv: srv.handle_request()
_f()
time.sleep(0.2) # wait for server to start

...so that we can send a request and print its response:

In [ ]:
send_recv('world', port, host) # TODO: should `send_recv` print result to stdout?

'Hello, world!\n'

In [ ]:
#|export
def fastdaemon_serve(cmd, port, host=None, timeout=None):
    "Serve `cmd` on `port`, with optional `host` and `timeout`"
    host = host or socket.gethostname()
    with DaemonServer((host,port), cmd, timeout=timeout) as srv:
        while not srv.handle_request(): pass

A convenient wrapper to instantiate and start a `DaemonServer` that handles requests until it's interrupted or times out. Here's the previous example using `fastdaemon_serve`:

In [ ]:
@threaded
def _f(): fastdaemon_serve(_cmd, 9999, timeout=1)
_f()
time.sleep(0.2) # wait for server to start

In [ ]:
send_recv('world', port, host)

'Hello, world!\n'

## Export -

In [ ]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()